In [4]:
import sys, os
from pathlib import Path
WORKDIR = Path('/home/alex/Programming/UCLA/CS188-DLCV-Project')
os.chdir(WORKDIR)
os.environ['WORKDIR'] = str(WORKDIR)
WORKDIR

PosixPath('/home/alex/Programming/UCLA/CS188-DLCV-Project')

In [15]:
!sudo apt install -yy htop ninja-build
#!pip install pytorch-lightning
!conda install pytorch-lightning=1 opencv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
htop is already the newest version (2.2.0-1+b1).
ninja-build is already the newest version (1.8.2-1).
The following package was automatically installed and is no longer required:
  python3-crcmod
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.



In [5]:
!git clone https://github.com/amikhalev/TinyHITNet.git TinyHITNet
HITNET = WORKDIR / 'TinyHITNet'
sys.path.append(str(HITNET))

fatal: destination path 'TinyHITNet' already exists and is not an empty directory.


In [6]:
!TORCH_CUDA_ARCH_LIST="3.7+PTX" pip install ./TinyHITNet/ext_op/

Processing ./TinyHITNet/ext_op
  Preparing metadata (setup.py) ... - done
  Created wheel for HitnetModule: filename=HitnetModule-0.0.0-cp37-cp37m-linux_x86_64.whl size=2399331 sha256=e25c2debfce7d7d7a43a3fef95a4216ee95a0d2ea707eb794fa3cce01f9c34e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-d0f0nyju/wheels/7c/53/75/d76e82895fa5974e0be9f13d627c1832ea50cf17245149ddbc
Successfully built HitnetModule


In [6]:
!nvidia-smi

Mon Mar 14 18:08:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   60C    P8     7W /  N/A |    178MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!DIR=kitti2015; mkdir -p $DIR; cd $DIR; \
wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip; \
unzip -qn data_scene_flow.zip
KITTI2015 = WORKDIR / "kitti2015"

--2022-03-14 18:10:15--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.169.165
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.169.165|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [10]:
!DIR=kitti2012; mkdir -p $DIR; cd $DIR; \
wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_stereo_flow.zip; \
unzip -qn data_stereo_flow.zip
KITTI2012 = WORKDIR /"kitti2012"

--2022-03-14 18:14:03--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_stereo_flow.zip
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.171.65
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.171.65|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [12]:
from preprocess import plane_fitting_kitti2012, plane_fitting_kitti2015
import multiprocessing as mp
import tqdm

def preprocess_main(root, list_path, module, n_gpu=1):
    root = Path(root)

    with open(list_path, "rt") as fp:
        file_list = [Path(line.strip()) for line in fp]

    lock_list = [mp.Lock() for _ in range(n_gpu)]
    with mp.Pool(n_gpu, module.process_init, [lock_list, root]) as pool:
        list(tqdm.tqdm(pool.imap_unordered(module.process, file_list), total=len(file_list)))

preprocess_main(KITTI2012 / 'training', HITNET / 'lists' / 'kitti2012_train.list', plane_fitting_kitti2012)
preprocess_main(KITTI2015 / 'training', HITNET / 'lists' / 'kitti2015_train.list', plane_fitting_kitti2015)

100%|██████████| 200/200 [02:05<00:00,  1.60it/s]


In [15]:
!cd {HITNET}; python3 train.py \
--log_dir logs \
--exp_name stereonet_kitti_new \
--model StereoNetNew \
--check_val_every_n_epoch 5 \
--max_steps 2000000 \
--max_disp 192 \
--max_disp_val 192 \
--optmizer Adam \
--lr 4e-4 \
--lr_decay 400000 0.25 408000 0.1 410000 0.025 \
--lr_decay_type Lambda \
--batch_size 1 \
--batch_size_val 1 \
--num_workers 2 \
--num_workers_val 2 \
--data_augmentation 0 \
--data_type_train KITTI2012 KITTI2015 \
--data_root_train {KITTI2012}/training {KITTI2015}/training \
--data_list_train lists/kitti2012_train170.list lists/kitti2015_train180.list \
--data_size_train 1152 320 \
--data_type_val KITTI2015 \
--data_root_val {KITTI2015}/training \
--data_list_val lists/kitti2015_val20.list \
--data_size_val 1242 375 \
--gpus -1 \
--accelerator gpu


Global seed set to 2021
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | model        | StereoNetNew     | 590 K 
1 | train_metric | MetricCollection | 0     
2 | val_metric   | MetricCollection | 0     
--------------------------------------------------
590 K     Trainable params
0         Non-trainable params
590 K     Total params
2.361     Total estimated model params size (MB)
Validation sanity check: 0it [00:00, ?it/s]/home/alex/.conda/envs/cs188-project/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve perfor